In [ ]:
#model selction tools
from sklearn.model_selection import train_test_split , GridSearchCV\
        , StratifiedKFold , TimeSeriesSplit,KFold,cross_val_score
#metrics 
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor,GradientBoostingRegressor
import xgboost as xgb
import catboost as cb
#tools
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline


from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor,GradientBoostingRegressor
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
import xgboost as xgb
import catboost as cb

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score




In [ ]:
import os
os.listdir("../input/bike-sharing-demand")

In [ ]:
import pandas as pd
import numpy as np

# train_path = r"train.csv"
# test_path = r"test.csv"
train_path = r"../input/bike-sharing-demand/train.csv"
test_path = r"../input/bike-sharing-demand/test.csv"
df_train = pd.read_csv(train_path,parse_dates=['datetime'],dayfirst=True)
df_test = pd.read_csv(test_path,parse_dates=['datetime'],dayfirst=True)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
def get_date_attributes(DF,Date_Col_name):
    
    DF['Year'] = DF[Date_Col_name].dt.year.astype('int')
    DF['Week'] = DF[Date_Col_name].dt.isocalendar().week.astype('int')
    DF['Month'] = DF[Date_Col_name].dt.month.astype('int')
    DF['WeekDay'] = DF[Date_Col_name].dt.weekday.astype('int')
    DF['Hour'] = DF['datetime'].dt.hour
    return DF

def new_feature(DF,function,new_col_name):
    DF[new_col_name]  = DF.apply(function,axis=1)
    return DF

def map_colmun_to_Categorical(DF,new_col_name,from_vals,to_vals):
    DF[new_col_name].replace(from_vals,to_vals,inplace=True)
    return DF
def CreateLag(DF,used_col,shift = -1):
    col_name = f'{used_col}_shift({shift})'
    DF[col_name] = DF[used_col].shift(shift)
    DF[col_name].fillna(0, inplace=True)
    return DF
def CreateRoll(DF,used_col,roll = 4):
    col_name = f'{used_col}_rolling({roll}With_Mean)'
    DF[col_name] = DF[used_col].rolling(roll).mean()
    DF[col_name].fillna(0, inplace=True)
    return DF
def CreateEWM(DF,used_col,com = 0.9):
    col_name = f'{used_col}_EWM_Com({com}With_Mean)'
    DF[col_name] = DF[used_col].ewm(com=com).mean()
    DF[col_name].fillna(0, inplace=True)
    return DF

In [ ]:
df_train = get_date_attributes(df_train,'datetime')
df_test = get_date_attributes(df_test,'datetime')

In [ ]:
def rush_hour(df):
    #from 8 am , 6 pm
    rush_range_1 = 2 # around the rush hour by +- 2 hrs
    rush_range_2 = 5 # around the rush hour by +- 4 hrs
    is_functional = df['workingday'] 
    
    if (abs(df['Hour'] - 8) <= rush_range_1): 
        return np.exp(-abs(df['Hour'] - 8)) * is_functional
    
    elif (abs(df['Hour'] - 18) <= rush_range_2):
        return np.exp(-abs(df['Hour'] - 18)) * is_functional
    else:
        return 0
        
def dead_hour(df):
    #dead_range_1 = 5 # around the dead hour by +- 3 hrs from 4 am
    is_functional = df['workingday'] 
    
    if(df['Hour'] in [22,23,0,1,2,3,4,5]):
        return np.exp(-abs(df['Hour'] - 4)) * is_functional
    else:
        return 0 

In [ ]:
day_state = lambda df : 1  if (df['Hour'] >=6 and df['Hour'] <=18) else 0

In [ ]:
Ideal = lambda df: 1 if (df['temp'] < 30 and df['windspeed'] > 30) else 0

In [ ]:
df_train = new_feature(df_train,rush_hour,'rush_hour')
df_train = new_feature(df_train,dead_hour,'dead_hour')
df_train = new_feature(df_train,day_state,'day_state')
df_train = new_feature(df_train,Ideal,'Ideal')

df_test = new_feature(df_test,rush_hour,'rush_hour')
df_test = new_feature(df_test,dead_hour,'dead_hour')
df_test = new_feature(df_test,day_state,'day_state')
df_test = new_feature(df_test,Ideal,'Ideal')

In [ ]:
ColsForLag =  ["temp" ,"weather" ,"rush_hour","dead_hour","humidity","windspeed","Ideal"]

t  = -1
for col in ColsForLag:
    df_train = CreateLag(df_train,col,(+t))
for col in ColsForLag:
    df_train = CreateLag(df_train,col,(+t-1))
for col in ColsForLag:
    df_train = CreateLag(df_train,col,(+t-2))
for col in  ColsForLag:
    df_train = CreateRoll(df_train,col,4)
for col in  ColsForLag:
    df_train = CreateEWM(df_train,col,0.9)
    
for col in ColsForLag:
    df_test = CreateLag(df_test,col,(+t))
for col in ColsForLag:
    df_test = CreateLag(df_test,col,(+t-1))
for col in ColsForLag:
    df_test = CreateLag(df_test,col,(+t-2))
for col in  ColsForLag:
    df_test = CreateRoll(df_test,col,4)
for col in  ColsForLag:
    df_test = CreateEWM(df_test,col,0.9)

In [ ]:
lag1_features , lag2_features , lag3_features , rolling_features,ewn_features = [],[],[],[],[]
t = -1

for used_col in ColsForLag:
    col_name = f'{used_col}_shift({+t})'
    lag1_features.append(col_name)
    
for used_col in ColsForLag:
    col_name = f'{used_col}_shift({+t-1})'
    lag2_features.append(col_name)
    
for used_col in ColsForLag:
    col_name = f'{used_col}_shift({+t-2})'
    lag3_features.append(col_name)

for used_col in ColsForLag:
    col_name = f'{used_col}_rolling({4}With_Mean)'
    rolling_features.append(col_name)

for used_col in ColsForLag:
    col_name = f'{used_col}_EWM_Com({0.9}With_Mean)'
    ewn_features.append(col_name)

In [ ]:
original_features = ['Hour','Week', 'Month','Year','windspeed'
                     ,'WeekDay','humidity','workingday','season','weather','Ideal']

weather_features=['rush_hour','dead_hour','day_state']

selected_features = original_features + weather_features + lag1_features + lag2_features + lag3_features + rolling_features + ewn_features


In [ ]:
df_train["count"] = np.log(df_train["count"]+0.00001)

In [ ]:
X1 = df_train[selected_features]
y1 = df_train["count"]

# tss = TimeSeriesSplit(n_splits=2)
# train_ind,test_ind  = tss.split(X1,groups=[20,1])

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.2, random_state=42,shuffle=True,)

In [ ]:
def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = 10**(y_true)
        y_pred = 10**(y_pred)
        
    log_true = np.nan_to_num(np.array([np.log(y+1.0) for y in y_true]))
    log_pred = np.nan_to_num(np.array([np.log(y+1.0) for y in y_pred]))
    
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    return output

rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False) 
Scoring = rmsle_scorer

In [ ]:
params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**params)
gbm_model.fit(X_train,y_train)

pred_test = gbm_model.predict(X_test)
pred_train= gbm_model.predict(X_train)

print('(Test) CatBoost Regression RMSLE:', rmsle(y_test, pred_test, True))
print('(Train) CatBoost Regression RMSLE:', rmsle(y_train, pred_train, True))

In [ ]:
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

# model = cb.CatBoostRegressor(loss_function='RMSE',random_state=0,max_depth=4,iterations=3200,
#                                            l2_leaf_reg=1,learning_rate=0.038,subsample=0.85)

model = cb.CatBoostRegressor(silent=True,loss_function='RMSE')
model.fit(X_train,y_train)
pred_test = model.predict(X_test)
pred_train= model.predict(X_train)

print('(Test) CatBoost Regression RMSLE:', rmsle(y_test, pred_test, True))
print('(Train) CatBoost Regression RMSLE:', rmsle(y_train, pred_train, True))


In [ ]:
df_test['y'] =  np.exp(model.predict(df_test[selected_features]))
final_df = df_test[['datetime', 'y']].copy()
save_to_path = r'submission.csv'
final_df.to_csv(save_to_path, index=False)